In [28]:
import cv2
import numpy as np
import csv
from PIL import Image

def merge_images(img1, img2):
    cloud_pixel = [255, 255, 255]
    merged = np.where(np.all(img1 == cloud_pixel, axis=-1)[:, :, np.newaxis], img2, img1)
    return merged

def get_centroid(contour):
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
    else:
        cx, cy = 0, 0
    return (cx, cy)

def distance(pt1, pt2):
    return np.sqrt((pt1[0] - pt2[0])**2 + (pt1[1] - pt2[1])**2)

def extract_positions_and_draw_circles(img, img_path):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    min_area = 0.001 * img.shape[0] * img.shape[1]
    filtered_contours = [contour for contour in contours if cv2.contourArea(contour) > min_area]
    sorted_contours = sorted(filtered_contours, key=cv2.contourArea, reverse=True)

    if len(sorted_contours) < 6:
        print(f"Unexpected number of contours: {len(sorted_contours)}")
        return []

    fence_centroid = get_centroid(sorted_contours[0])
    print(fence_centroid)
    animal_contours = [contour for contour in sorted_contours[1:6] if distance(get_centroid(contour), fence_centroid) > 50]
    positions = []

    for contour in animal_contours:
        cx, cy = get_centroid(contour)
        positions.append((cx, cy))
        cv2.circle(img, (cx, cy), 20, (0, 255, 0), 2)

    modified_img_path = img_path.replace("_sample0.png", "_marked.png")
    cv2.imwrite(modified_img_path, img)
    return positions

predictions = []

for i in range(1000):
    img_path1 = f"level_06/test_data/field{i:03d}_sample0.png"
    img_path2 = f"level_06/test_data/field{i:03d}_sample1.png"
    img1 = np.asarray(Image.open(img_path1).convert('RGB'))
    img2 = np.asarray(Image.open(img_path2).convert('RGB'))
    merged_img = merge_images(img1, img2)
    positions = extract_positions_and_draw_circles(merged_img, img_path1)

    flattened_positions = [coord for position in positions for coord in position]
    predictions.append(flattened_positions)

with open('predictions.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(predictions)


(165, 53)
(35, 22)
(52, 42)
(157, 92)
(156, 96)
(76, 160)
(166, 163)
(35, 99)
(179, 150)
(56, 131)
(114, 76)
(156, 176)
(158, 84)
(53, 62)
(107, 84)
(175, 134)
(62, 166)
(28, 105)
(68, 127)
(68, 156)
(179, 108)
(30, 107)
(35, 149)
(31, 90)
(20, 32)
(90, 181)
(144, 100)
(36, 174)
(94, 89)
(73, 26)
(173, 86)
(79, 163)
(93, 168)
(163, 139)
(76, 162)
(87, 178)
(118, 168)
(164, 176)
(30, 123)
(155, 149)
(97, 31)
(88, 47)
(141, 167)
(169, 168)
(23, 173)
(93, 162)
(177, 88)
(79, 90)
(91, 162)
(28, 44)
(178, 140)
(103, 83)
(32, 175)
(34, 65)
(105, 143)
(59, 81)
(45, 46)
(127, 130)
(165, 77)
(58, 120)
(44, 30)
(139, 156)
(91, 120)
(150, 115)
(66, 37)
(101, 75)
(56, 34)
(72, 52)
(132, 128)
(136, 116)
(148, 162)
(131, 169)
(176, 119)
(169, 165)
(31, 158)
(26, 43)
(120, 104)
(67, 112)
(45, 129)
(159, 142)
(166, 175)
(145, 78)
(37, 60)
(144, 177)
(96, 146)
(29, 141)
(104, 159)
(50, 64)
(176, 142)
(104, 142)
(113, 171)
(80, 142)
(20, 145)
(74, 124)
(93, 117)
(48, 140)
(96, 157)
(170, 153)
(160, 149)

NameError: name 'fence_centroid' is not defined